# Ingeniería de datos

En este notebook se documenta el proceso de las tareas asociadas al rol de ingeniería de datos

## Librerías

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Carga de datos

In [42]:
df = pd.read_csv("..\Data\Raw_data.csv", low_memory=False)

## Tarea 2: Selección, limpieza y alistamiento de datos 

In [43]:
df.head()

,periodo,estu_tipodocumento,estu_consecutivo,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_depto_ubicacion,...,fami_tienecomputador,fami_tieneinternet,fami_tienelavadora,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global
0,20181,CC,SB11201810003214,URBANO,N,B,ACADÉMICO,376001002376,376001002376,76,...,Si,Si,Si,B1,78.0,68.0,76,69,67,353
1,20181,CC,SB11201810003214,URBANO,N,B,ACADÉMICO,376001002376,376001002376,76,...,Si,Si,Si,B1,78.0,68.0,76,69,67,353
2,20181,TI,SB11201810043861,RURAL,NaN,B,ACADÉMICO,476520003791,476520003791,76,...,Si,Si,Si,A-,41.0,49.0,41,49,44,227
3,20181,TI,SB11201810043861,RURAL,NaN,B,ACADÉMICO,476520003791,476520003791,76,...,Si,Si,Si,A-,41.0,49.0,41,49,44,227
4,20181,CC,SB11201810004937,RURAL,N,OTRO,ACADÉMICO,425175032542,425175032542,25,...,Si,Si,Si,A2,59.0,57.0,48,57,55,273


In [44]:
columnas_numericas = df.select_dtypes(include=['number']).columns.tolist()
columnas_no_numericas = df.select_dtypes(exclude=['number']).columns.tolist()
print(f"Columnas no numéricas: {columnas_no_numericas}\nColumnas numéricas: {columnas_numericas}")

Columnas no numéricas: ['estu_tipodocumento', 'estu_consecutivo', 'cole_area_ubicacion', 'cole_bilingue', 'cole_calendario', 'cole_caracter', 'cole_depto_ubicacion', 'cole_genero', 'cole_jornada', 'cole_mcpio_ubicacion', 'cole_naturaleza', 'cole_nombre_establecimiento', 'cole_nombre_sede', 'cole_sede_principal', 'estu_depto_presentacion', 'estu_depto_reside', 'estu_estadoinvestigacion', 'estu_estudiante', 'estu_fechanacimiento', 'estu_genero', 'estu_mcpio_presentacion', 'estu_mcpio_reside', 'estu_nacionalidad', 'estu_pais_reside', 'estu_privado_libertad', 'fami_cuartoshogar', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda', 'fami_personashogar', 'fami_tieneautomovil', 'fami_tienecomputador', 'fami_tieneinternet', 'fami_tienelavadora', 'desemp_ingles']
Columnas numéricas: ['periodo', 'cole_cod_dane_establecimiento', 'cole_cod_dane_sede', 'cole_cod_depto_ubicacion', 'cole_cod_mcpio_ubicacion', 'cole_codigo_icfes', 'estu_cod_depto_presentacion', 'estu_cod_mcpio_presen

In [45]:
# Información básica del DataFrame
print(f"El dataset tiene {df.shape[0]} filas y {df.shape[1]} columnas")
print("\nInformación del DataFrame:")
df.info()

El dataset tiene 32348 filas y 51 columnas

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32348 entries, 0 to 32347
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   periodo                        32348 non-null  int64  
 1   estu_tipodocumento             32348 non-null  object 
 2   estu_consecutivo               32348 non-null  object 
 3   cole_area_ubicacion            32348 non-null  object 
 4   cole_bilingue                  27513 non-null  object 
 5   cole_calendario                32348 non-null  object 
 6   cole_caracter                  31074 non-null  object 
 7   cole_cod_dane_establecimiento  32348 non-null  int64  
 8   cole_cod_dane_sede             32348 non-null  int64  
 9   cole_cod_depto_ubicacion       32348 non-null  int64  
 10  cole_cod_mcpio_ubicacion       32348 non-null  int64  
 11  cole_codigo_icfes              3234

### Limpieza y transformación

In [48]:
df_clean = df.copy()

In [47]:
# Buscar valores que NO coincidan con el formato dd/mm/yyyy
valores_raros = df[~df['estu_fechanacimiento'].astype(str).str.match(r'^\d{2}/\d{2}/\d{4}$')]['estu_fechanacimiento']

if not valores_raros.empty:
    print("Valores inesperados en 'estu_fechanacimiento':")
    print(valores_raros.unique())
else:
    print("Todos los valores en 'estu_fechanacimiento' son fechas válidas.")

Valores inesperados en 'estu_fechanacimiento':
['ESTUDIANTE' 'PUBLICAR']


Situación identificada:

En los registros del colegio __"INST SER INTERNACIONAL COMUNA 17"__, se detectó un desplazamiento de columnas hacia la derecha a partir de estu_fechanacimiento, causado muy probablemente por un error en la estructura del archivo CSV original (ej: una coma adicional o un salto de línea mal formado). Como consecuencia:

Los campos de información personal (como género y municipio) estaban desplazados, haciendo que estu_genero contuviera fechas de nacimiento y estu_mcpio_presentacion contuviera géneros.

Acciones tomadas:

* Se reubicaron manualmente los valores en sus columnas correctas para los registros afectados.


In [49]:
# Filtrar registros del colegio problemático
mask = df_clean['cole_nombre_establecimiento'] == 'INST SER INTERNACIONAL COMUNA 17'
df_problem = df_clean[mask].copy()

# Corregir desplazamiento (mover todas las columnas desde 'estu_fechanacimiento' una posición a la izquierda)
columnas_desplazadas = [
    'estu_fechanacimiento', 'estu_genero', 'estu_mcpio_presentacion',
    'estu_mcpio_reside', 'estu_nacionalidad', 'estu_pais_reside',
    'estu_privado_libertad', 'fami_cuartoshogar', 'fami_educacionmadre',
    'fami_educacionpadre', 'fami_estratovivienda', 'fami_personashogar',
    'fami_tieneautomovil', 'fami_tienecomputador', 'fami_tieneinternet',
    'fami_tienelavadora', 'desemp_ingles']

# Mover cada columna una posición a la izquierda
for i in range(len(columnas_desplazadas) - 1):
    df_problem[columnas_desplazadas[i]] = df_problem[columnas_desplazadas[i+1]]

# # Asignar NaN a la última columna desplazada (punt_global ya no tiene dato original)
# df_problem['punt_global'] = np.nan

# Reemplazar los registros corregidos en el DataFrame original
df_clean[mask] = df_problem

# Verificar corrección
print(df_clean[mask][['estu_fechanacimiento', 'estu_genero', 'punt_global']].head())

     estu_fechanacimiento estu_genero  punt_global
1272           25/03/1998           M           44
1273           25/03/1998           M           44
1607           15/11/2000           F           53
1608           15/11/2000           F           53
1760           04/10/2000           M           46


In [50]:
df_clean['estu_fechanacimiento'] = pd.to_datetime(
    df_clean['estu_fechanacimiento'], 
    format='%d/%m/%Y',  
    errors='coerce'
)

In [51]:
fechas_invalidas_df = df_clean[df_clean['estu_fechanacimiento'].isna()]
fechas_invalidas_df.head()

,periodo,estu_tipodocumento,estu_consecutivo,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_depto_ubicacion,...,fami_tienecomputador,fami_tieneinternet,fami_tienelavadora,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global
21371,20181,TI,SB11201810008367,URBANO,N,B,NaN,376001043897,376001043897,76,...,Si,Si,Si,A1,49.0,42.0,45,37,46,215
23573,20181,CC,SB11201810010858,URBANO,NaN,A,ACADÉMICO,317174000755,317174000755,17,...,3 a 4,No,Si,Si,NaN,NaN,46,38,55,52
27074,20181,CC,SB11201810010855,URBANO,NaN,A,ACADÉMICO,317174000755,317174000755,17,...,3 a 4,No,Si,Si,NaN,NaN,40,29,48,46
30772,20181,TI,SB11201810014960,URBANO,S,B,ACADÉMICO,311769003474,311769003474,11,...,Si,Si,Si,B+,81.0,100.0,70,76,76,403
30773,20181,TI,SB11201810014960,URBANO,S,B,ACADÉMICO,311769003474,311769003474,11,...,Si,Si,Si,B+,81.0,100.0,70,76,76,403


In [ ]:
# Extraer año y semestre de 'periodo'
df_clean['semestre_prueba'] = df_clean['periodo'].astype(str).str[-1].astype(int)

In [ ]:
df_clean.head()

### Nulos

#### Identificación

In [6]:
nulos_por_columna = df.isnull().sum()
print("Valores nulos por columna:")
print(nulos_por_columna)

Valores nulos por columna:
periodo                             0
estu_tipodocumento                  0
estu_consecutivo                    0
cole_area_ubicacion                 0
cole_bilingue                    4835
cole_calendario                     0
cole_caracter                    1274
cole_cod_dane_establecimiento       0
cole_cod_dane_sede                  0
cole_cod_depto_ubicacion            0
cole_cod_mcpio_ubicacion            0
cole_codigo_icfes                   0
cole_depto_ubicacion                0
cole_genero                         0
cole_jornada                        0
cole_mcpio_ubicacion                0
cole_naturaleza                     0
cole_nombre_establecimiento         0
cole_nombre_sede                    0
cole_sede_principal                 0
estu_cod_depto_presentacion       137
estu_cod_mcpio_presentacion         2
estu_cod_reside_depto              16
estu_cod_reside_mcpio              16
estu_depto_presentacion             0
estu_depto_reside      

In [7]:
# Porcentaje de valores faltantes por columna
missing_values = df.isnull().mean() * 100
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

print("\nPorcentaje de valores faltantes por columna:")
print(missing_values)



Porcentaje de valores faltantes por columna:
cole_bilingue                  14.946828
fami_estratovivienda            5.688141
fami_educacionpadre             5.422283
fami_educacionmadre             5.388277
fami_tieneinternet              5.388277
cole_caracter                   3.938420
fami_personashogar              3.109930
fami_tieneautomovil             2.720415
fami_cuartoshogar               2.516384
fami_tienecomputador            2.442191
fami_tienelavadora              2.371089
estu_cod_depto_presentacion     0.423519
punt_ingles                     0.423519
estu_cod_reside_depto           0.049462
estu_depto_reside               0.049462
estu_mcpio_reside               0.049462
estu_cod_reside_mcpio           0.049462
desemp_ingles                   0.009274
estu_cod_mcpio_presentacion     0.006183
punt_matematicas                0.006183
dtype: float64


#### Manejo

__Alternativa 1__ 
* Para variables categóricas: Imputar la moda
* Para variables numéricas: Imputar la media

In [9]:
# # Imputar valores faltantes en columnas categóricas
# for col in columnas_no_numericas:
#     if df_clean[col].isnull().any():
#         mode_val = df_clean[col].mode()[0]
#         df_clean[col].fillna(mode_val, inplace=True)

# # Imputar valores faltantes en columnas numéricas
# for col in columnas_numericas:
#     if df_clean[col].isnull().any():
#         median_val = df_clean[col].median()
#         df_clean[col].fillna(median_val, inplace=True)

# # Verificar que no quedan valores faltantes
# print("\nValores faltantes después de la limpieza:")
# print(df_clean.isnull().sum().sum())

__Alternativa 2__
* Para variables categóricas: Reemplazar nulos por "Desconocido"
* Para variables numéricas: Mantener los nulos

In [10]:
df_clean[columnas_no_numericas] = df_clean[columnas_no_numericas].fillna('Desconocido')

Se prefiere la alternativa 2 pues:  

En este estudio de los resultados Saber 11, preservar los nulos en variables numéricas (como puntajes) evita distorsionar métricas clave, ya que estos valores faltantes podrían reflejar casos reales (ej: estudiantes que no presentaron una prueba específica). Para las categóricas, usar "Desconocido" en lugar de la moda es más transparente: diferencia claramente entre datos no reportados (que pueden ser significativos para análisis socioeducativos) y categorías genuinas. Este enfoque mantiene la integridad estadística de los puntajes numéricos sin perder información valiosa en las categóricas, crucial para un análisis educativo donde el contexto importa.

In [11]:
print("\nResumen de nulos después de limpieza:")
print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])


Resumen de nulos después de limpieza:
estu_cod_depto_presentacion    137
estu_cod_mcpio_presentacion      2
estu_cod_reside_depto           16
estu_cod_reside_mcpio           16
punt_ingles                    137
punt_matematicas                 2
dtype: int64


Ante estos resultados de nulos remanentes parece tentador eliminar dichas filas pues:  

Dado que se tienen 32348 filas en total, estos son los porcentajes de nulos por columna problemática:

_estu_cod_depto_presentacion_: 137 nulos (0.42% del total)  
_estu_cod_mcpio_presentacion_: 2 nulos (0.006%)  
_estu_cod_reside_depto_$/mcpio: 16 nulos cada una (0.05%)  
_punt_ingles_: 137 nulos (0.42%)  
_punt_matematicas_: 2 nulos (0.006%)  

In [12]:
# Filtrar filas con nulos
nulos_ingles = df_clean[df_clean['punt_ingles'].isnull()]

# Distribución por departamento del colegio
print("\n\t DISTRIBUCIÓN GEOGRÁFICA (punt_ingles) \t")
print("Top 3 departamentos con nulos:")
print(nulos_ingles['cole_depto_ubicacion'].value_counts(normalize=True).head(3))
print("\nDistribución general (dataset completo):")
print(df_clean['cole_depto_ubicacion'].value_counts(normalize=True).head(3))

# Relación con área urbana/rural
print("\n\t ÁREA URBANA/RURAL ")
print("Proporción en nulos vs total:")
print("Nulos:", nulos_ingles['cole_area_ubicacion'].value_counts(normalize=True))
print("Total:", df_clean['cole_area_ubicacion'].value_counts(normalize=True))

# Rendimiento académico asociado
print("\n\t RENDIMIENTO ACADÉMICO ")
print(f"Puntaje global promedio (nulos): {nulos_ingles['punt_global'].mean():.1f}")
print(f"Puntaje global promedio (total): {df_clean['punt_global'].mean():.1f}")

# Solapamiento con otras columnas
print("\n\t CRUCE CON OTRAS VARIABLES ")
print("Registros con nulos en punt_matematicas:", 
      len(nulos_ingles[nulos_ingles['punt_matematicas'].isnull()]))
print("Distribución por tipo de colegio:")
print(nulos_ingles['cole_naturaleza'].value_counts(normalize=True))


	 DISTRIBUCIÓN GEOGRÁFICA (punt_ingles) 	
Top 3 departamentos con nulos:
cole_depto_ubicacion
VALLE     0.985401
CALDAS    0.014599
Name: proportion, dtype: float64

Distribución general (dataset completo):
cole_depto_ubicacion
VALLE           0.437956
BOGOTA          0.231575
CUNDINAMARCA    0.066032
Name: proportion, dtype: float64

	 ÁREA URBANA/RURAL 
Proporción en nulos vs total:
Nulos: cole_area_ubicacion
URBANO    1.0
Name: proportion, dtype: float64
Total: cole_area_ubicacion
URBANO    0.873037
RURAL     0.126963
Name: proportion, dtype: float64

	 RENDIMIENTO ACADÉMICO 
Puntaje global promedio (nulos): 49.6
Puntaje global promedio (total): 299.1

	 CRUCE CON OTRAS VARIABLES 
Registros con nulos en punt_matematicas: 2
Distribución por tipo de colegio:
cole_naturaleza
NO OFICIAL    1.0
Name: proportion, dtype: float64


In [13]:
# Filtrar filas con nulos
nulos_depto = df_clean[df_clean['estu_cod_depto_presentacion'].isnull()]

# Distribución por departamento del colegio
print("\n\t DISTRIBUCIÓN GEOGRÁFICA ")
print("Top 3 departamentos (cole_depto_ubicacion) con nulos:")
print(nulos_depto['cole_depto_ubicacion'].value_counts(normalize=True).head(3))

# Comparación con distribución general
print("\nDistribución general (dataset completo):")
print(df_clean['cole_depto_ubicacion'].value_counts(normalize=True).head(3))

# Relación con área urbana/rural
print("\n\t ÁREA URBANA/RURAL ")
print("Proporción en nulos vs total:")
print("Nulos:", nulos_depto['cole_area_ubicacion'].value_counts(normalize=True))
print("Total:", df_clean['cole_area_ubicacion'].value_counts(normalize=True))

# Solapamiento con municipio
print("\n\t SOLAPAMIENTO CON MUNICIPIO ")
solapamiento = len(nulos_depto[nulos_depto['estu_cod_mcpio_presentacion'].isnull()])
print(f"Registros con nulos en DEPTO y MUNICIPIO: {solapamiento}/{len(nulos_depto)}")


	 DISTRIBUCIÓN GEOGRÁFICA 
Top 3 departamentos (cole_depto_ubicacion) con nulos:
cole_depto_ubicacion
VALLE     0.985401
CALDAS    0.014599
Name: proportion, dtype: float64

Distribución general (dataset completo):
cole_depto_ubicacion
VALLE           0.437956
BOGOTA          0.231575
CUNDINAMARCA    0.066032
Name: proportion, dtype: float64

	 ÁREA URBANA/RURAL 
Proporción en nulos vs total:
Nulos: cole_area_ubicacion
URBANO    1.0
Name: proportion, dtype: float64
Total: cole_area_ubicacion
URBANO    0.873037
RURAL     0.126963
Name: proportion, dtype: float64

	 SOLAPAMIENTO CON MUNICIPIO 
Registros con nulos en DEPTO y MUNICIPIO: 2/137


⚠️ El análisis reveló que los valores nulos en _punt_ingles_ y _estu_cod_depto_presentacion_ presentan patrones claros: el 98.5% se concentran en el departamento de Valle (frente al 43.8% esperado), específicamente en colegios urbanos no oficiales, con puntajes globales significativamente más bajos (49.6 vs 299.1). Esta distribución no aleatoria sugiere que los nulos corresponden a casos válidos (ej: estudiantes que no presentaron la prueba) o limitaciones en la recolección de datos en esta región. Por lo tanto, se decidió conservar los nulos en lugar de eliminarlos, ya que su remoción sesgaría la representatividad del análisis para el Valle y ocultaría un hallazgo potencialmente relevante sobre la participación en las pruebas. Esta decisión preserva la integridad de los patrones geográficos y académicos en los datos.